### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

## Importing Python modules for the demo

In [ ]:
import requests
import os, time, uuid

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

## Make sure you have the correct Training Endpoint, Key and Resource ID

In [ ]:
TRAINING_ENDPOINT = "ENTER TRAINING ENDPOINT HERE"
training_key = "ENTER TRAINING RESOURCE KEY HERE"
training_resource_id = "ENTER TRAINING RESOURCE ID HERE"

In [ ]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

### Getting a list of all Custom Vision projects 

In [ ]:
all_project = trainer.get_projects()

In [ ]:
for each_project in all_project:
    print("Project ID: ",each_project.id)
    print("Project Name: ",each_project.name)

### Select the project ID that you are interested in exporting. Make sure the project type is set to "Compact".

In [ ]:
project_id = all_project[0].id
print("Project ID: ", project_id)

In [ ]:
all_iterations = trainer.get_iterations(project_id)

In [ ]:
for index, each_iteration in enumerate(all_iterations):
    print("|--------- Iteration [{}] --------".format(index))
    print("Iteration ID: ",each_iteration.id)
    print("Iteration Name: ",each_iteration.name)
    print("Iteration Exportable: ",each_iteration.exportable)
    print("-----------------------------------------------------")

### Above please verify that your project-> Iteration must be exportalble
- Iteration Exportable: True

Make sure there are exportable iterations. If none of the iteration is exportable, follow these steps:
* Go to the Custom Vision portal
* Select a project and change your project setting to support the compact domain
* Create another iteration for model training

You can check the *"Azure Custom Vision Portal Demo 5: Export a Model"* video for how to perform these steps.

Then you can rerun the cells above.

In [ ]:
for index, each_iteration in enumerate(all_iterations):
    model_perf = trainer.get_iteration_performance(project_id, each_iteration.id)
    print("----- Model Perormance {} -------- ".format(index))
    print("Iteration ID: ",each_iteration.id)
    print("Precision: {}   Recall: {} -------- ".format(model_perf.precision, model_perf.recall))
    print("-----------------------------------------------------")

### Select the project ID that you are interested in exporting. Make sure it is exportable.

In [ ]:
selected_iteration_id = all_iterations[0].id
print(selected_iteration_id)

### Platform Choices are
- CoreML, TensorFlow, DockerFile, ONNX, VAIDK, and OpenVino

### Flavor Choices are
- Linux, Windows, ONNX10, ONNX12, ARM, TensorFlowNormal, and TensorFlowLite.

### raw = True|False
- If True, you can get raw JSON response along with the object model response

# Start the model export process

In [ ]:
platform = "TensorFlow"
flavor = "TensorFlowLite"

In [ ]:
export_process = trainer.export_iteration(project_id, selected_iteration_id, platform, flavor, raw=True)

In [ ]:
print(export_process.output)

In [ ]:
print(export_process.output.status)

In [ ]:
### Code snippet is from Azure SDK and Documentation
### https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
### This step may take a long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project_id, selected_iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

In [ ]:
print(export_process.output.status)

In [ ]:
print(export_process.output.download_uri)

In [ ]:
## Downloading the model from url
if export_process.output.status == "Done":
    # Ready to Download
    model_export_file = requests.get(export_process.output.download_uri)
    with open("birds_flowers_model_tensorflow.zip", "wb") as file:
        file.write(model_export_file.content)

# Collecting Export Status by project ID and iteration ID

In [ ]:
exports_offline = trainer.get_exports(project_id, selected_iteration_id)

In [ ]:
for e in exports_offline:
    print(e)

In [ ]:
for e in exports_offline:
    if e.status == 'Done':
        print("Downloading model from URL: ", e.download_uri)
        model_export_file = requests.get(e.download_uri)
        with open("birds_flowers_model_tensorflow.zip", "wb") as file:
            file.write(model_export_file.content)

In [ ]:
!ls -l birds_flowers_model_tensorflow.zip